In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import h5py
import boto.s3
import glob
import boto3
from zipfile import ZipFile
import shutil
import gcsfs

In [3]:
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF = pd.read_csv('gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.05-wheelchair-0.5-20220917/inexus/sfbay_accessibility_fleet_pct-0.05_2010__20220913.csv.gz', compression = 'gzip', dtype = dtypes)

C:\Users\nazanin\AppData\Local\Temp\2\ipykernel_1240\3123155120.py:22: DtypeWarning: Columns (59,61,63,68,70,75,84,86,88,90,91,97,103,107,111,115,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,151,156,157,158,159,160,161,162,163,164,181,184,187,188,189,191,193,196,197,199,201,221,222,223,224,233,234) have mixed types. Specify dtype option on import or set low_memory=False.
  eventsSF = pd.read_csv('gs://beam-core-outputs/sfbay-2010-accessibility-fleetpct-0.05-wheelchair-0.5-20220917/inexus/sfbay_accessibility_fleet_pct-0.05_2010__20220913.csv.gz', compression = 'gzip', dtype = dtypes)


In [6]:
eventsSF.head()

,Unnamed: 0.1,IDMerged,tripIndex,actStartTime,actEndTime,duration_travelling,cost_BEAM,actStartType,actEndType,duration_walking,duration_in_privateCar,duration_on_bike,duration_in_ridehail,distance_travelling,duration_in_transit,distance_walking,distance_bike,distance_ridehail,distance_privateCar,distance_transit,vehicleIds_estimate,mode_choice_planned_BEAM,mode_choice_actual_BEAM,vehicleIds,numPassengers,distance_mode_choice,replanning_status,reason,parkingType,transit_bus,transit_subway,transit_tram,transit_cable_car,ride_hail_pooled,transit_rail,fuelFood,fuelElectricity,fuelBiodiesel,fuelDiesel,fuel_not_Food,fuelGasoline,fuel_marginal,BlockGroupStart,BlockGroupEnd,emissionFood,emissionElectricity,emissionDiesel,emissionGasoline,emissionBiodiesel,emission_marginal,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6,trip_id,person_id,household_id,tour_id,primary_purpose_x,trip_num,outbound,trip_count,purpose,destination_x,origin_x,destination_logsum_x,depart,trip_mode_AS_trips,logsum_trip_mode_AS_trips,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination_y,origin_y,household_id_x,tdd,start,end,duration,composition,destination_logsum_y,tour_mode_AS_tours,logsum_tours_mode_AS_tours,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose_y,Unnamed: 0,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,DIS,in_wheelchair,TAZ_x,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ_y,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,BIKE,WALK_LOC,WALK_LRF,WALK_EXP,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,0,1,577.0,29708.0,29548.0,160.0,0.000000,othmaint,Home,0.0,160.0,0.0,0.0,4313.862,0.0,0.000,0.0,0.0,4313.862,0.0,"body-1,teleportationSharedVehicle-1-1,body-1",hov2_teleportation,hov2_teleportation,NaN,NaN,4313.862,0,NaN,NaN,0,0,0,0,0,0,0.000,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,641.0,583.0,0.0,0.0,0.0,0.000000,0.0,0.0,160.0,0.0,Home_to_othmaint,car,car,577.0,1.0,1735309.0,72.0,sc

#### Read the files directly from S3

In [2]:
%%time
s3 = boto3.client("s3")
key = "output/sfbay/sfbay-pilates-50pop-20veh__2022-07-20_21-48-54_xwf/ITERS/it.0/2.events.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF = pd.read_csv(obj['Body'], compression = 'gzip', dtype = dtypes)

<timed exec>:25: DtypeWarning: Columns (2,28,33,34,37,42,47,54,58,61) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 3min 9s
Wall time: 3min 9s


In [83]:
%%time
s3 = boto3.client("s3")
key = "output/sfbay/sfbay-pilates-50pop-20veh__2022-07-20_21-48-54_xwf/ITERS/it.0/0.events.csv.gz"  #the path should be updated
obj = s3.get_object(Bucket="beam-outputs", Key=key)
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object",
    'primaryFuelType': "category",
    'secondaryFuelType': 'category',
    'currentTourMode': 'category',
    'currentActivity': 'category',
    'nextActivity': 'category'    
}
eventsSF0 = pd.read_csv(obj['Body'], compression = 'gzip', dtype = dtypes)

<timed exec>:25: DtypeWarning: Columns (2,28,33,34,37,42,47,54,58,61) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 6min 26s
Wall time: 6min 27s


In [97]:
eventsSF0.reason.unique()

array([nan, 'ResourceUnavailable RIDE_HAIL_POOLED',
       'ResourceUnavailable RIDE_HAIL',
       'MissedTransitPickup DRIVE_TRANSIT',
       'ResourceUnavailable RIDE_HAIL_TRANSIT',
       'ResourceCapacityExhausted WALK_TRANSIT',
       'MissedTransitPickup RIDE_HAIL_TRANSIT',
       'ResourceCapacityExhausted RIDE_HAIL_TRANSIT',
       'ResourceCapacityExhausted DRIVE_TRANSIT',
       'ResourceCapacityExhausted BIKE_TRANSIT'], dtype=object)

In [98]:
eventsSF0.vehicle.unique()

array([nan, 'SF:7590302', 'SF:7606357', ..., '694097', '165582', '771968'],
      dtype=object)

In [5]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
# Showing the entire number in dataframe
pd.set_option('float_format', '{:f}'.format)

In [5]:
# Adding scenario info
eventsSF['scenario'] = "ridehail"
eventsSF['scenario'] = eventsSF['scenario'].astype("category")
eventsSF['lever'] = "access_fleetsz"
eventsSF['lever'] = eventsSF['lever'].astype("category")
eventsSF['year'] = 2018
eventsSF['lever_position'] = "0.2_0.5"
eventsSF['lever_position'] = eventsSF['lever_position'].astype("category")

In [6]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [7]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [8]:
# shift column 'person' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [9]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [10]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [11]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [12]:
# Shift column 'IDMerged' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [13]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

CPU times: total: 1min 48s
Wall time: 1min 48s


In [14]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [15]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [16]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [17]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [18]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

CPU times: total: 4min 23s
Wall time: 4min 23s


In [19]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [20]:
#IDnan = eventsSF[eventsSF['IDMerged'].isna()]
#IDnan.shape

In [21]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [22]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

CPU times: total: 17 s
Wall time: 17 s


In [23]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [24]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [25]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [26]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [27]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [28]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [29]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [30]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [31]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [32]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

#### Adding the census blocks

In [33]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'):
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    df.drop(columns=[xcol, ycol], inplace=True)
    return df.loc[~df.index.duplicated(keep='first'), :]

In [34]:
# census_2010 = "https://beam-core-act.s3.amazonaws.com/deepDive/RawData/Census_Blocks/Scenario_2010_shp/"
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')

In [35]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'startX', 'startY', 'BlockGroupStart')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 56min 50s
Wall time: 56min 50s


In [36]:
%%time
eventsSF = addGeometryIdToDataFrame(eventsSF, BGs, 'endX', 'endY', 'BlockGroupEnd')

C:\Users\nazanin\.conda\envs\geo_env\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


CPU times: total: 56min 50s
Wall time: 56min 50s


#### Adding new columns

In [37]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [38]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [39]:
eventsSF['duration_travelling'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [40]:
eventsSF['distance_travelling'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [41]:
eventsSF['distance_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [42]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['duration_travelling'], np.nan)

In [43]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['distance_travelling'], np.nan)

In [44]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['duration_travelling'], np.nan)

In [45]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['distance_travelling'], np.nan)

In [46]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['duration_travelling'], np.nan)

In [47]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [48]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['duration_travelling'], np.nan)

In [49]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['distance_travelling'], np.nan)

In [50]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['duration_travelling'], np.nan)

In [51]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['distance_travelling'], np.nan)

In [52]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [53]:
#eventsSF = eventsSF.set_index('IDMerged')

In [54]:
eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                eventsSF['primaryFuel'], np.nan)

In [55]:
eventsSF['emissionFood'] = eventsSF['fuelFood'] * 8.3141841e-9 * 0

In [56]:
eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                eventsSF['primaryFuel'], np.nan)

In [57]:
eventsSF['emissionElectricity'] = eventsSF['fuelElectricity'] * 2.77778e-10 * 947.2 * 0.0005

In [58]:
eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [59]:
eventsSF['emissionDiesel'] = eventsSF['fuelDiesel'] * 8.3141841e-9 * 10.180e-3

In [60]:
eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                eventsSF['primaryFuel'], np.nan)

In [61]:
eventsSF['emissionBiodiesel'] = eventsSF['fuelBiodiesel'] * 8.3141841e-9 * 10.180e-3

In [62]:
eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            , eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [63]:
eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [64]:
eventsSF['emissionGasoline'] = eventsSF['fuelGasoline'] * 8.3141841e-9 * 8.89e-3

In [65]:
# Marginal fuel
conditions  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled'), 
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),
               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')]
choices = [eventsSF['fuel_not_Food']/eventsSF['numPassengers'], 0 , eventsSF['fuelFood'], eventsSF['fuel_not_Food']]

In [66]:
eventsSF['fuel_marginal'] = np.select(conditions, choices, default=np.nan)

In [67]:
# Marginal emission
conditions1  = [(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelElectricity'].notna() != 0), 
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelGasoline'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelBiodiesel'].notna() != 0),
               (eventsSF['modeBEAM_rh'] == 'ride_hail_pooled') & (eventsSF['fuelDiesel'].notna() != 0),             
               (eventsSF['modeBEAM_rh'] == 'walk_transit') | (eventsSF['modeBEAM_rh'] == 'drive_transit')|
               (eventsSF['modeBEAM_rh'] == 'ride_hail_transit')|(eventsSF['modeBEAM_rh'] == 'bus')|(eventsSF['modeBEAM_rh'] == 'subway')|
               (eventsSF['modeBEAM_rh'] == 'rail')|(eventsSF['modeBEAM_rh'] == 'tram')|(eventsSF['modeBEAM_rh'] == 'cable_car')|
               (eventsSF['modeBEAM_rh'] == 'bike_transit'),

               (eventsSF['modeBEAM_rh'] == 'walk')|(eventsSF['modeBEAM_rh'] == 'bike'),
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelElectricity'].notna() != 0),
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelGasoline'].notna() != 0),           
              
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelBiodiesel'].notna() != 0),   
               
               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelDiesel'].notna() != 0),

               (eventsSF['modeBEAM_rh'] == 'ride_hail')|(eventsSF['modeBEAM_rh'] == 'car')| 
               (eventsSF['modeBEAM_rh'] == 'car_hov2')| (eventsSF['modeBEAM_rh'] == 'car_hov3')|
               (eventsSF['modeBEAM_rh'] == 'hov2_teleportation')| (eventsSF['modeBEAM_rh'] == 'hov3_teleportation')&
               (eventsSF['fuelFood'].notna() != 0)]

choices1 = [eventsSF['emissionElectricity']/eventsSF['numPassengers'],
           eventsSF['emissionGasoline']/eventsSF['numPassengers'],
           eventsSF['emissionBiodiesel']/eventsSF['numPassengers'],
           eventsSF['emissionDiesel']/eventsSF['numPassengers'],           
           0 , 
           eventsSF['emissionFood'], 
           eventsSF['emissionElectricity'],
           eventsSF['emissionGasoline'],
           eventsSF['emissionBiodiesel'],
           eventsSF['emissionDiesel'],
           eventsSF['emissionFood']]

In [68]:
eventsSF['emission_marginal'] = np.select(conditions1, choices1, default=np.nan)

In [69]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [70]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

#### Trip Index

In [71]:
#eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripId.fillna(method='ffill')

#### Mode Choice planned and actual

In [72]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('last')

CPU times: total: 25.2 s
Wall time: 25.2 s


In [73]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripId', 'type'])['modeBEAM'].transform('first')

CPU times: total: 25.2 s
Wall time: 25.2 s


In [74]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [75]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [76]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [77]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [78]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

CPU times: total: 1.73 s
Wall time: 1.73 s


In [79]:
eventsSF['transit_bus'] = np.where(eventsSF['modeBEAM_rh']=='bus', 1, 0)
eventsSF['transit_subway'] = np.where(eventsSF['modeBEAM_rh']=='subway', 1, 0)
eventsSF['transit_tram'] = np.where(eventsSF['modeBEAM_rh']=='tram', 1, 0)
eventsSF['transit_rail'] = np.where(eventsSF['modeBEAM_rh']=='rail', 1, 0)
eventsSF['transit_cable_car'] = np.where(eventsSF['modeBEAM_rh']=='cable_car', 1, 0)

In [80]:
eventsSF['ride_hail_pooled'] = np.where(eventsSF['modeBEAM_rh']=='ride_hail_pooled', 1, 0)

In [81]:
eventsSF.head()

,IDMerged,person,driver,riders,vehicle,secondaryFuelLevel,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,price,primaryFuelLevel,time,type,link,facility,actType,modeBEAM,incentive,tollCost,cost_BEAM,currentTourMode,arrivalTime,departureTime,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,shiftStatus,parkingZoneId,fuel,duration,vehicleType,legMode,cost,departTime,requireWheelchair,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,links,numPassengers,primaryFuel,reason,score,scenario,lever,year,lever_position,modeBEAM_rh,BlockGroupStart,BlockGroupEnd,actEndTime,actStartTime,duration_travelling,distance_travelling,distance_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,fuelFood,emissionFood,fuelElectricity,emissionElectricity,fuelDiesel,emissionDiesel,fuelBiodiesel,emissionBiodiesel,fuel_not_Food,fuelGasoline,emissionGasoline,fuel_marginal,emission_marginal,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status,transit_bus,transit_subway,transit_tram,transit_rail,transit_cable_car,ride_hail_pooled
0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30999.000000,ModeChoice,NaN,NaN,NaN,hov2_teleportation,NaN,NaN,NaN,hov2_teleportation,NaN,NaN,NaN,HOV2_TELEPORTATION,56792.000000,true,3650.361000,1.000000,"WALK,CAR_HOV2,WALK","body-1,teleportationSharedVehicle-1-1,body-1",Home,othmaint,default,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ridehail,access_fleetsz,2018,0.2_0.5,hov2_teleportation,NaN,NaN,NaN,NaN,NaN,3650.361000,3650.361000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3650.361000,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,default,hov2_teleportation,hov2_teleportation,0,0,0,0,0,0,0
1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30999.000000,actend,56792.000000,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ridehail,access_fleetsz,2018,0.2_0.5,NaN,NaN,NaN,30999.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,default,NaN,NaN,0,0,0,0,0,0,0
2,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30999.000000,departure,56792.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hov2_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ridehail,access_fleetsz,2018,0.2_0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,default,NaN,NaN,0,0,0,0,0,0,0
3,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31136.000000,TeleportationEvent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hov2_teleportation,31136.000000,30999.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ridehail,access_fleetsz,2018,0.2_0.5,hov2_teleportation,060855035091,060855033053,NaN,NaN,137.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,default,NaN,NaN,0,0,0,0,0,0,0
4,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31136.000000,arrival,75436.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ridehail,access_fleetsz,2018,0.2_0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [92]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 
            'actEndTime': np.sum, 
            'duration_travelling': np.sum, 
            'cost_BEAM': np.sum, 
            'actStartType': np.sum, 
            'actEndType': np.sum, 
            'duration_walking': np.sum, 
            'duration_in_privateCar': np.sum, 
            'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 
            'distance_travelling': np.sum, 
            'duration_in_transit': np.sum, 
            'distance_walking': np.sum, 
            'distance_bike': np.sum, 
            'distance_ridehail': np.sum, 
            'distance_privateCar': np.sum, 
            'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'distance_mode_choice': np.sum,
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str))),
            'parkingType': lambda x: ', '.join(list(x.dropna().astype(str))),
            'transit_bus': np.sum, 
            'transit_subway': np.sum, 
            'transit_tram': np.sum, 
            'transit_cable_car': np.sum,
            'ride_hail_pooled': np.sum, 
            'transit_rail': np.sum,
            'year': lambda x: ', '.join(set(x.dropna().astype(str))),
            'lever_position': lambda x: ', '.join(set(x.dropna().astype(str))),
            'scenario': lambda x: ', '.join(set(x.dropna().astype(str))),
            'fuelFood': np.sum, 
            'fuelElectricity': np.sum, 
            'fuelBiodiesel': np.sum, 
            'fuelDiesel': np.sum, 
            'fuel_not_Food': np.sum, 
            'fuelGasoline': np.sum, 
            'fuel_marginal': np.sum,
            'BlockGroupStart': 'first',
            'BlockGroupEnd': 'last',
            'emissionFood': np.sum, 
            'emissionElectricity': np.sum, 
            'emissionDiesel': np.sum, 
            'emissionGasoline': np.sum,
            'emissionBiodiesel': np.sum, 
            'emission_marginal': np.sum,
            'lever': lambda x: ', '.join(set(x.dropna().astype(str)))
           }).reset_index() 

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

CPU times: total: 14min 35s
Wall time: 14min 35s


In [93]:
Person_Trip_eventsSF['duration_door_to_door'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [94]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['duration_door_to_door'] - Person_Trip_eventsSF['duration_travelling'] 

In [95]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [96]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"vehicleIds_estimate"}, inplace=True) 

In [97]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [98]:
# Column with five summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car']

In [99]:
Person_Trip_eventsSF['mode_choice_actual_5'] = np.select(conditions, choices, default=np.nan)

In [100]:
# Column with six summarized modes
conditions  = [(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_pooled'), 
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk_transit') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'drive_transit')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike_transit'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'walk'), (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'bike'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car') | (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov2')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'car_hov3')|(Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov2_teleportation')| (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'hov3_teleportation'),
               (Person_Trip_eventsSF['mode_choice_actual_BEAM'] == 'ride_hail_transit')]
choices = [ 'ride_hail', 'transit', 'walk', 'bike', 'car', 'ride_hail_transit']

In [101]:
Person_Trip_eventsSF['mode_choice_actual_6'] = np.select(conditions, choices, default=np.nan)

In [102]:
Person_Trip_eventsSF = Person_Trip_eventsSF.drop(Person_Trip_eventsSF[Person_Trip_eventsSF.duration_door_to_door < 0].index)

In [ ]:
#Person_Trip_eventsSF.to_csv('C:/Shared-Work/Data/CleanData/sf_2018_base_core_act.csv', index = False)

In [103]:
Person_Trip_eventsSF.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Ridehail_Accessibility/sf_rh_access_0.2_fleetsz_0.5_BEAM.csv', index=False)  #the path should be updated

#### Merging with ActivitySim files

In [90]:
s3 = boto3.client("s3")
key = "deepDive/CleanData/SanFrancisco/Ridehail_Accessibility/sf_rh_access_0.2_fleetsz_0.5_BEAM.csv"
obj = s3.get_object(Bucket="beam-core-act", Key=key)
Person_Trip_eventsSF = pd.read_csv(obj['Body'])

KeyboardInterrupt: 

In [91]:
# Read the GitHub files 
households = pd.read_csv('C:/Shared-Work/Data/sf_rh_wc_ASim/households.csv.gz',compression = 'gzip')
persons = pd.read_csv('C:/Shared-Work/Data/sf_rh_wc_ASim/persons.csv.gz',compression = 'gzip')
trips = pd.read_csv('C:/Shared-Work/Data/sf_rh_wc_ASim/trips.csv.gz',compression = 'gzip')

In [92]:
# Merge BEAM households and persons 
persons = persons.sort_values(by=['household_id']).reset_index(drop=True)
households = households.sort_values(by=['household_id']).reset_index(drop=True)
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [93]:
# Merge trips, households and persons
trips = trips.sort_values(by=['person_id']).reset_index(drop=True)
hhpersons = hhpersons.sort_values(by=['person_id']).reset_index(drop=True)
hhperTrips = pd.merge(left=trips, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [94]:
hhperTrips.head()

,trip_id,person_id,household_id_x,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum,Unnamed: 0,earning,worker,student,hispanic.1,person_sex,PNUM,sex,household_id_y,hours,MAR,person_age,edu,school_zone_id,race_id,race,work_at_home,age,work_zone_id,p_hispanic,relate,hispanic,ptype,pemploy,pstudent,home_x,home_y,age_16_to_19,age_16_p,adult,male,female,has_non_worker,has_retiree,has_preschool_kid,has_driving_kid,has_school_kid,has_full_time,has_part_time,has_university,student_is_employed,nonstudent_to_school,is_student,is_gradeschool,is_highschool,is_university,school_segment,is_worker,home_taz,value_of_time,school_taz,distance_to_school,roundtrip_auto_time_to_school,workplace_taz,workplace_location_logsum,distance_to_work,workplace_in_cbd,work_taz_area_type,roundtrip_auto_time_to_work,work_auto_savings,work_auto_savings_ratio,free_parking_at_work,cdap_activity,travel_active,under16_not_at_school,has_preschool_kid_at_home,has_school_kid_at_home,mandatory_tour_frequency,work_and_school_and_worker,work_and_school_and_student,num_mand,num_work_tours,num_joint_tours,non_mandatory_tour_frequency,num_non_mand,num_escort_tours,num_eatout_tours,num_shop_tours,num_maint_tours,num_discr_tours,num_social_tours,num_non_escort_tours,in_wheelchair,gt55,seniors,VEHICL,lcm_county_id,hh_children,block_id,gt2,hispanic_head,age_of_head,race_of_head,tenure_mover,hh_size,sf_detached,tenure,hh_cars,income,hh_age_of_head,serialno,num_workers,hh_race_of_head,hh_income,recent_mover,hh_workers,hispanic_status_of_head,hh_seniors,hhsize,hh_type,TAZ,HHT,sample_rate,chunk_id,income_in_thousands,income_segment,median_value_of_time,hh_value_of_time,num_non_workers,num_drivers,num_adults,num_children,num_young_children,num_children_5_to_15,num_children_16_to_17,num_college_age,num_young_adults,non_family,family,home_is_urban,home_is_rural,auto_ownership,hh_work_auto_savings_ratio,num_under16_not_at_school,num_travel_active,num_travel_active_adults,num_travel_active_preschoolers,num_travel_active_children,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours
0,1517,4,865,189,othdiscr,1,False,1,Home,1005,1127,NaN,16.000000,DRIVEALONEFREE,-1.893187,186953.000000,0.000000,0.000000,0.000000,0.000000,female,4.000000,2.000000,1735309.000000,0.000000,5.000000,19 and under,0.000000,650.000000,6.000000,asian,0.000000,0.000000,-1.000000,no,2.000000,0.000000,8.000000,4.000000,1.000000,-121.806440,37.350304,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1.000000,False,641.000000,19.798008,652.000000,3.387731,0.040000,-1.000000,NaN,NaN,False,NaN,0.000000,0.000000,0.000000,False,M,True,False,False,False,school1,False,False,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,0.000000,0.000000,2.000000,6085.000000,yes,60855035091006.000000,1.000000,no,35.000000,6.000000,own not recent,four or more,yes,1.000000,two or more,240000.000000,gt35-lt65,2010000482865.000000,2.000000,asian,gt150,0.000000,two or more,0.000000,no,4.000000,3.000000,0.000000,4.000000,0.097000,60138.000000,240.000000,4.000000,12.860000,29.682171,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,True,False,False,False,2.000000,1.550681,0.000000,4.000000,2.000000,2.000000,2.000000,2.000000,True,0_tours,0.000000
1,1513,4,865,189,othdiscr,1,True,1,othdiscr,1127,1005,NaN,15.000000,DRIVEALONEFREE,-1.304090,186953.000000,0.000000,0.000000,0.000000,0.000000,female,4.000000,2.000000,1735309.000000,0.000000,5.000000,19 and under,0.000000,650.000000,6.000000,asian,0.000000,0.000000,-1.000000,no,2.000000,0.000000,8.000000,4.000000,1.000000,-121.806440,37.350304,False,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True,True,False,False,1.000000,False,641.000000,19.798008,652.000000,3.387731,0.040000,-1.0000

In [10]:
Person_Trip_eventsSF.head()

,IDMerged,tripIndex,BlockGroupEnd,BlockGroupStart,actEndTime,actEndType,actStartTime,actStartType,cost_BEAM,distance_bike,...,transit_rail,transit_subway,transit_tram,vehicleIds,year,duration_door_to_door,waitTime,actPurpose,mode_choice_actual_5,mode_choice_actual_6
0,1,default,6.085503e+10,6.085504e+10,117250.0,Homeothmaintschool,117891.0,othmaintschoolHome,0.000000,0.0,...,0,0,0,NaN,2018,641.0,0.0,Homeothmaintschool_to_othmaintschoolHome,car,car
1,2,default,6.085504e+10,6.085503e+10,248485.0,Homeworkatworkworkothmaint,251376.0,workatworkworkothmaintHome,8.308248,0.0,...,0,0,0,"539011, body-2",2018,2891.0,0.0,Homeworkatworkworkothmaint_to_workatworkworkot...,car,car
2,3,default,6.085503e+10,6.085504e+10,208437.0,HomeworkHomeworkwork,211638.0,workHomeworkworkHome,6.076459,0.0,...,0,0,0,"539012, body-3",2018,3201.0,0.0,HomeworkHomeworkwork_to_workHomeworkworkHome,car,car
3,4,default,6.085503e+10,6.085504e+10,166707.0,Homeschoolescort,167604.0,schoolescortHome,0.000000,0.0,...,0,0,0,NaN,2018,897.0,0.0,Homeschoolescort_to_schoolescortHome,car,car
4,86,default,6.081607e+10,6.081607e+10,232588.0,HomeothdiscrHomeshoppingothmaint,234319.0,othdiscrHomeshoppingothmaintHome,3.829256,0.0,...,0,0,0,"body-86, 725511",2018,1731.0,0.0,HomeothdiscrHomeshoppingothmaint_to_othdiscrHo...,car,car


In [89]:
hhperTrips.head()

NameError: name 'hhperTrips' is not defined

In [8]:
# Merge person_trip level BEAM with activity sim merged files
hhperTrips = hhperTrips.sort_values(by=['person_id', 'trip_id']).reset_index(drop=True)
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripIndex']).reset_index(drop=True)
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=hhperTrips, how='left', left_on=["IDMerged", 'tripIndex'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [156]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [157]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [158]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [159]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [160]:
# Save the output file to S3
eventsASim.to_csv('s3://beam-core-act/deepDive/CleanData/SanFrancisco/Ridehail_Accessibility/sf_rh_access_0.2_fleetsz_0.5.csv', index=False)  #the path should be updated

CPU times: total: 6min 17s
Wall time: 8min 1s


In [46]:
# Delete the utilities files downloaded and saved in the system
os.remove('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice.zip')   #the path should be updated
shutil.rmtree('C:/Users/nazanin/Documents/beam-core-analysis/Users/Nazanin/trip_mode_choice')   #the path should be updated